In [5]:
import cv2
import numpy as np

In [6]:
def convert_grey(frame: np.ndarray) -> np.ndarray:
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return gray_frame


def display_green_color(frame: np.ndarray, kernel_: tuple) -> np.ndarray:
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([85, 255, 255])

    green_mask = cv2.inRange(hsv_frame, lower_green, upper_green)

    kernel = np.ones(kernel_, np.uint8)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)

    comb_frame = np.vstack((frame, np.repeat(green_mask[:, :, np.newaxis], 3, axis=2)))
    return comb_frame


In [7]:
def reSize_frame(frame, scale):
    return cv2.resize(frame, (-1, -1), fx=scale, fy=scale)

In [8]:
!pip install patched-yolo-infer


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from ultralytics import YOLO
from patched_yolo_infer import (visualize_results_usual_yolo_inference)

In [10]:
imgsz = 640
conf = 0.4
iou = 0.7

model = YOLO("yolov8m-seg.pt")

100%|██████████| 52.4M/52.4M [00:54<00:00, 1.01MB/s]


In [11]:
model2 = YOLO("yolov8m.pt")

100%|██████████| 49.7M/49.7M [00:54<00:00, 956kB/s] 


In [12]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Ошибка! Не удается открыть камеру.")
else:
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    while True:
        # Читаем кадр с камеры
        ret, frame = cap.read()
        
        if not ret:
            break

        #frame = convert_grey(frame)
        #frame = display_green_color(frame, (10, 10))

        '''
        frame = visualize_results_usual_yolo_inference(
            frame, 
            model, 
            imgsz, 
            conf, 
            iou,
            segment=True, 
            delta_colors=1, 
            thickness=6,
            font_scale=1.5, 
            show_boxes=False, 
            fill_mask=True,
            alpha=0.35,
            show_confidences=False,
            return_image_array=True
        )
        '''
        
        frame = visualize_results_usual_yolo_inference(
            frame, 
            model2, 
            imgsz, 
            conf, 
            iou,
            segment=False, 
            delta_colors=1, 
            thickness=6,
            font_scale=1.5, 
            show_boxes=True, 
            fill_mask=True,
            alpha=0.35,
            show_confidences=False,
            return_image_array=True
        )


        frame = reSize_frame(frame, 0.5)
            
        # Отображаем изображение
        cv2.imshow('Webcam', frame)
    
        # Выходим по нажатию клавиши 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Освобождаем ресурсы и закрываем окна
cap.release()
cv2.destroyAllWindows()